In [52]:
import pandas as pd
import numpy as np
import os.path
root_path = os.path.dirname(os.getcwd())

# Import video data (thanks Cowen & Keltner!)
video_data = pd.read_csv(os.path.join(root_path, 'DATA/video_data.csv'))

,Filename,Admiration,Adoration,Aesthetic Appreciation,Amusement,Anger,Anxiety,Awe,Awkwardness,Boredom,...,vitality,vulnerability,warmth,weakness,weariness,wonder,worry,wrath,yearning,zeal
0,0001.mp4,0.000000,0.0,0.083333,0.00000,0.083333,0.083333,0.083333,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0
1,0002.mp4,0.000000,0.0,0.000000,0.25000,0.000000,0.000000,0.000000,0.000000,0.083333,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0
2,0003.mp4,0.083333,0.0,0.083333,0.66667,0.000000,0.000000,0.000000,0.083333,0.000000,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0,0.0
3,0004.mp4,0.000000,0.0,0.000000,0.75000,0.000000,0.000000,0.000000,0.083333,0.000000,...,0.0,0.0,0.0,0.0,0.11111,0.0,0.00000,0.0,0.0,0.0
4,0005.mp4,0.000000,0.0,0.000000,0.00000,0.000000,0.333330,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.11111,0.0,0.0,0.0


In [55]:
# Separate out 34 emotion category scores, downcase column names
# Fraction of respondents who chose from multiple-select
video_category_data = video_data.iloc[:, 1:35]
video_category_data.index = video_data['Filename']

# Separate out 14 affective dimension scores
# 1-9 ratings
video_dimension_data = video_data.iloc[:, 35:49]
video_dimension_data.index = video_data['Filename']

# Separate 600 free response term scores
# Fraction of respondants who chose from multiple-select dropdown
video_term_data = video_data.iloc[:, 49:]
video_term_data.index = video_data['Filename']

# Select distinct category scores
# Subset of category scores
video_distinct_category_data = video_data[[
    'Admiration',
    'Adoration',
    'Aesthetic Appreciation',
    'Amusement',
    'Anger',
    'Anxiety',
    'Awe',
    'Awkwardness',
    'Boredom',
    'Calmness',
    'Confusion',
    'Craving',
    'Disgust',
    'Empathic Pain',
    'Entrancement',
    'Excitement',
    'Fear',
    'Horror',
    'Interest',
    'Joy',
    'Nostalgia',
    'Relief',
    'Romance',
    'Sadness',
    'Satisfaction',
    'sexual desire',
    'Surprise'
]]
video_distinct_category_data.index = video_data['Filename']

In [56]:
# Data supported matrices (v_to_c.dot(video_vector) = other_vector)
v_to_c = video_category_data.transpose()
v_to_d = video_dimension_data.transpose()
v_to_t = video_term_data.transpose()
v_to_dc = video_distinct_category_data.transpose()

In [57]:
# Inverse matrices
c_to_v = pd.DataFrame(np.linalg.pinv(v_to_c.values), v_to_c.columns, v_to_c.index)
d_to_v = pd.DataFrame(np.linalg.pinv(v_to_d.values), v_to_d.columns, v_to_d.index)
t_to_v = pd.DataFrame(np.linalg.pinv(v_to_t.values), v_to_t.columns, v_to_t.index)
dc_to_v = pd.DataFrame(np.linalg.pinv(v_to_dc.values), v_to_dc.columns, v_to_dc.index)

In [58]:
# Category inference matrices
c_to_d = v_to_d.dot(c_to_v)
c_to_t = v_to_t.dot(c_to_v)
c_to_dc = v_to_dc.dot(c_to_v)

In [59]:
# Dimension inference matrices
d_to_c = v_to_c.dot(d_to_v)
d_to_t = v_to_t.dot(d_to_v)
d_to_dc = v_to_dc.dot(d_to_v)

In [60]:
# Term inference matrices
t_to_c = v_to_c.dot(t_to_v)
t_to_d = v_to_d.dot(t_to_v)
t_to_dc = v_to_dc.dot(t_to_v)

In [61]:
# Distinct category inference matrices
dc_to_c = v_to_c.dot(dc_to_v)
dc_to_d = v_to_d.dot(dc_to_v)
dc_to_t = v_to_t.dot(dc_to_v)

In [62]:
# SLOW IF FIRST TIME
# Install and import Spacy natural language utility
!pip install spacy
import spacy

# Load large english language model
!python -m spacy download en_core_web_md
nlp = spacy.load('en_core_web_lg')

tensorflow 1.11.0 has requirement setuptools<=39.1.0, but you'll have setuptools 40.4.3 which is incompatible.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

    Linking successful
    /home/russell/anaconda3/lib/python3.6/site-packages/en_core_web_md -->
    /home/russell/anaconda3/lib/python3.6/site-packages/spacy/data/en_core_web_md

    You can now load the model via spacy.load('en_core_web_md')



In [ ]:
# Install and import spelling corrector
!pip install autocorrect
from autocorrect import spell

# Define function to normalize phrase
def normalize(text):
    return ' '.join([spell(word) for word in text.split()])

# Define function to lemmatize phrase
def lemmatize(text):
    return ' '.join([token.lemma_ for token in nlp(text)])

In [ ]:
# Create dataframe for terms, lemmas, docs
term_data = pd.DataFrame([])
term_data['term'] = video_term_data.columns
term_data['term_lemma'] = term_data['term'].apply(lemmatize)
term_data['term_doc'] = term_data['term_lemma'].apply(nlp)
term_data.head()

In [ ]:
# NOT USED
import math

def normalize_vector(vector):
    magnitude = math.sqrt((vector * vector).sum())
    print(magnitude)
    return vector / magnitude

In [ ]:
# Define function to get term vector from phrase
def get_term_vector(phrase, n_terms):
    phrase_normal = normalize(phrase)
    phrase_lemma = lemmatize(phrase_normal)
    phrase_doc = nlp(phrase_lemma)
    
    term_vector = term_data['term_doc'].apply(phrase_doc.similarity)
    term_vector.index = term_data['term']
    
    top_terms = term_vector.sort_values(ascending=False).head(n_terms).index
    term_vector[~term_vector.index.isin(top_terms)] = 0
    return term_vector

In [ ]:
def interpret_phrase(phrase, n_terms):
    term_vector = get_term_vector(phrase, n_terms)
    print('FREE RESPONSE TERMS:\n', term_vector.sort_values(ascending=False).head(n_terms), '\n')
    
    video_vector = t_to_v.dot(term_vector)
    print('VIDEOS:\n', video_vector.sort_values(ascending=False).head(), '\n')
    
    category_vector = v_to_c.dot(video_vector)
    print('DISTINCT CATEGORIES:\n', category_vector.sort_values(ascending=False), '\n')
    
    dimension_vector = v_to_d.dot(video_vector) 
    print('AFFECTIVE DEMENSIONS:\n', dimension_vector, '\n')
    
    distinct_category_vector = v_to_dc.dot(video_vector)
    print('DISTINCT CATEGORIES:\n', distinct_category_vector.sort_values(ascending=False), '\n')


In [ ]:
# QUALITY CHECK
term_vector = get_term_vector("I'm feeling pissed", 3)
print(term_vector.sort_values(ascending=False).head())
video_vector = t_to_v.dot(term_vector)
print(video_vector.sort_values(ascending=False).head())
print(v_to_t.dot(video_vector).sort_values(ascending=False).head())

In [53]:
interpret_phrase("happiness", 3)

FREE RESPONSE TERMS:
 term
happiness            1.000000
extreme happiness    0.817343
joy                  0.775831
Name: term_doc, dtype: float64 

VIDEOS:
 1744    0.304397
1348    0.176554
1187    0.161230
1695    0.134951
2137    0.129371
dtype: float64 

DISTINCT CATEGORIES:
 Joy                       0.949334
Adoration                 0.621104
Amusement                 0.542459
Awe                       0.409756
Admiration                0.263923
Excitement                0.150091
Calmness                  0.141120
Nostalgia                 0.133603
Sympathy                  0.126788
Triumph                   0.076548
Interest                  0.052251
Envy                      0.046074
Aesthetic Appreciation    0.044872
Pride                     0.038585
Entrancement              0.032333
Surprise                  0.024513
Romance                   0.017221
Boredom                   0.015355
Guilt                     0.011599
Sadness                   0.011041
Fear             

In [ ]:
#TAKEWAYS
# Not great
# doesn't reliably find best match
# term dataset covers a very broad range but what I need is more nuance on common feelings
# could try universal sentence encoder, but I think that'll be slow and not much better

# THINGS TO TRY NEXT
# Training a learning model to extract terms effectively
# Assuming this is a+ and moving on to how I'd use it (to give me more info on what I'll need)